# **P**redicting **A**ccurate **S**pectra **T**races for **A**strophysical **SOSS** Spectra Demo (WORK IN PROGRESS)
----
This notebook demos how to use the `PASTASOSS` package to rapidly generate the trace positions for NIRISS/SOSS observations.

#### About this Notebook
**_Author_**: Tyler Baines, STScI Science Support Analyst

**_Date Updated_**: 5/1/2023

**_Observatory_**: JWST

**_Instrument/Mode_**: NIRISS/SOSS



## Imports

In [1]:
import pastasoss
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'pastasoss'

## Generate Trace Positions

Get the traces positions for a given pupil wheel positions angle "PWCPOS" for a given spectral order(s).

In [ ]:
#Arbritray gr700xd pwcpos angle
pwcpos = 245.89 

traces_order1 = get_trace_from_reference_transform(pwcpos=pwcpos, order='1', interp=True)
traces_order2 = get_trace_from_reference_transform(pwcpos=pwcpos, order='2', interp=True)

# or you can comment out the lines above and use the following:
# traces_order1, traces_order2 = pastasoss.get_trace_from_reference_transform(pwcpos=pwcpos, order='12', interp=True)

## Plot traces

In [ ]:
x1, y1 = traces_order1
x2, y2, = traces_order2

plt.figure()
plt.title(f'gr700xd trace w/ PWCPOS={pwcpos}')
plt.plot(x1,y1, label=f'order 1')
plt.plot(x2,y2, label=f'order 2')
plt.legend()
plt.show()